In [2]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time
import tqdm

# third-party imports
import numpy as np
import pandas as pd
import requests

# environment variables
from dotenv import load_dotenv
load_dotenv()
ANYDEALS_API_KEY = os.getenv("ANYDEAL_KEY")
# customisations - ensure tables show all columns
# pd.set_option("max_columns", 100)
print(ANYDEALS_API_KEY)

d0754faa36ea0d0d87c737908e13de1b56e857e5


In [12]:
# merge all the game data into one dataframe
temp_df = pd.DataFrame()
for i in range(0, 3000,10):
    if i == 0:
        temp_df = pd.read_csv(f"textfiles/game_info/last_successful_batch_{i}_game_info_anydeal.csv", index_col=0)
    else:
        temp_df = pd.concat([temp_df, pd.read_csv(f"textfiles/game_info/last_successful_batch_{i}_game_info_anydeal.csv", index_col=0)])
temp_df.to_csv('anydeal.csv')

In [13]:
# merge all the game prices into one dataframe
temp_df = pd.DataFrame()
for i in range(0, 3000,10):
    if i == 0:
        temp_df = pd.read_csv(f"textfiles/price_history/last_successful_batch_{i}_price_history.csv", index_col=0)
    else:
        temp_df = pd.concat([temp_df, pd.read_csv(f"textfiles/price_history/last_successful_batch_{i}_price_history.csv", index_col=0)])

temp_df.to_csv('anydeal_prices.csv')

In [14]:
df_game_info = pd.read_csv('anydeal.csv')
df_price_history = pd.read_csv('anydeal_prices.csv')
df_price_history.rename(columns={'appid':'id'}, inplace=True)

# merge the two dataframes

df = pd.merge(df_game_info, df_price_history, on='id', how='inner')

In [19]:
df_without_history = df[df['price_history'] == '[]']
df_without_history.to_csv('anydeal_without_history.csv')